In [13]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from movies.models import Movie
from django.conf  import settings
import pandas as pd

ratings_path = settings.DATA_DIR / "ratings_small.csv" 
# qs = Movie.objects.all().values("title", "release_date", "rating_avg", "rating_count")
# df = pd.DataFrame(qs)
# df.head(n=10)
df = pd.read_csv(ratings_path)
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
from django.contrib.auth import get_user_model
User = get_user_model()


In [33]:
current_users = User.objects.all().values_list("id", flat = True)
rating_users = df["userId"].tolist()

In [34]:
missing_user_ids = set(rating_users)-set(current_users)
for uid in missing_user_ids:
    User.objects.create(id = uid, username=f"user number {uid}")

In [42]:
import math
from decimal import Decimal
df['value'] = df['rating'].map(lambda x:math.ceil(Decimal(x)))
df.head()



,userId,movieId,rating,timestamp,value
0,1,31,2.5,1260759144,3
1,1,1029,3.0,1260759179,3
2,1,1061,3.0,1260759182,3
3,1,1129,2.0,1260759185,2
4,1,1172,4.0,1260759205,4


In [47]:
df['user_id'] = df['userId']
df['object_id'] = df['movieId']

In [44]:
df.head()


,userId,movieId,rating,timestamp,value,user_id,object_id
0,1,31,2.5,1260759144,3,1,31
1,1,1029,3.0,1260759179,3,1,1029
2,1,1061,3.0,1260759182,3,1,1061
3,1,1129,2.0,1260759185,2,1,1129
4,1,1172,4.0,1260759205,4,1,1172


In [49]:
clean_df = df.filter(['user_id', 'value', 'object_id'], axis=1)
clean_df.head()

,user_id,value,object_id
0,1,3,31
1,1,3,1029
2,1,3,1061
3,1,2,1129
4,1,4,1172


In [62]:
from  ratings.models import Rating

In [71]:
Rating.objects.all().delete()

In [72]:
from django.contrib.contenttypes.models import ContentType
ctype = ContentType.objects.get(app_label = 'movies', model='movie')

In [73]:
new_ratings = []
rating_records = clean_df.to_dict('records')
rating_records
for r in rating_records:
    r['content_type'] = ctype
    new_ratings.append(Rating(**r))
Rating.objects.bulk_create(new_ratings, ignore_conflicts=True, batch_size=1000)

In [77]:
from ratings.tasks import task_update_movie_ratings
task_update_movie_ratings()